# Creating CREATE File from Data

In [1]:
pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.9.0-py3-none-any.whl size=259467 sha256=371563675b3f7e241d601447117202fcf0f78e6dced8c53bb107f291add82e35
  Stored in directory: /home/jovyan/.cache/pip/wheels/5a/6a/b3/89ab0efac04cf204cfc65f2bff4776865cfe5ff2e1b94ebaff
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

## Tweets from popular people

In [3]:
tweets_df = pd.read_csv('tweets.csv')

In [4]:
tweets_df

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655
...,...,...,...,...,...,...,...,...,...,...
52537,ddlovato,Life couldn't be better right now. 😊,NaN,06/01/2015 23:10,5.526030e+17,en,NaN,NaN,32799,23796
52538,ddlovato,First Monday back in action. I'd say 21.6 mile...,NaN,06/01/2015 02:17,5.522880e+17,en,NaN,NaN,21709,12511
52539,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",NaN,05/01/2015 03:42,5.519470e+17,en,NaN,NaN,25269,15583
52540,ddlovato,❄️ http://t.co/sHCFdPpGPa,NaN,05/01/2015 00:06,5.518920e+17,und,NaN,NaN,15985,10456


## User connections by IDs

In [5]:
relations_file = open('twitter_combined.txt', 'r')
lines = relations_file.readlines()

relations = []
users = set()
# Strips the newline character
for line in lines:
    line = line.replace('\n','')
    id1, id2 = line.split(' ')
    users.add(id1)
    users.add(id2)
    relations.append([id1, id2])

In [6]:
relations

[['214328887', '34428380'],
 ['17116707', '28465635'],
 ['380580781', '18996905'],
 ['221036078', '153460275'],
 ['107830991', '17868918'],
 ['151338729', '222261763'],
 ['19705747', '34428380'],
 ['222261763', '88323281'],
 ['19933035', '149538028'],
 ['158419434', '17434613'],
 ['149538028', '153226312'],
 ['364971269', '153226312'],
 ['100581193', '279787626'],
 ['113058991', '69592091'],
 ['151338729', '187773078'],
 ['406628822', '262802533'],
 ['460282402', '88323281'],
 ['280935165', '437804658'],
 ['222261763', '27633075'],
 ['285312927', '151338729'],
 ['279787626', '131613362'],
 ['158419434', '17675120'],
 ['394263193', '100581193'],
 ['254839786', '88323281'],
 ['204317520', '21548772'],
 ['67864340', '172883064'],
 ['270449528', '297801196'],
 ['153226312', '187773078'],
 ['67864340', '8088112'],
 ['153226312', '17116707'],
 ['394263193', '14925700'],
 ['124528830', '307458983'],
 ['204317520', '160237722'],
 ['220368467', '54228724'],
 ['206923844', '103598216'],
 ['15322

In [7]:
['214328887', '34428380'] in relations

True

In [8]:
len(relations)

2420766

In [9]:
len(users)

81306

## Make CSV Files

### Users

In [10]:
header = "userId:ID,name,:Label\n"
body = ""
count = 0
for user in users:
    body += user + ",user" + str(count) + ",USER\n"
    count += 1
csv_content = header + body[:-1]

In [11]:
f = open("users.csv", "w")
f.write(csv_content)
f.close()

### Follows

In [12]:
f = open("relations.csv", "w")
f.write(":START_ID,:END_ID,:TYPE\n")
for relation in relations:
    f.write(relation[0] + "," + relation[1] + ",FOLLOWS\n")
f.close()

## Queries

In [13]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [14]:
conn = Neo4jConnection(uri="bolt://graph_db:7687", user="neo4j", pwd="password")

### 25 Users who follow the most users

In [19]:
results = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN a, COUNT(b) as following ORDER BY following DESC LIMIT 25")

In [20]:
for record in results:
    print(record['following'])

### 25 Users with most followers

In [21]:
results = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 25")

In [22]:
for record in results:
    print(record['followers'])